<a href="https://colab.research.google.com/github/DJCordhose/practical-llm/blob/main/Assessment_Llama_3.1_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assessment on LLama 3.1 8B

https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct

## Measurements

### 4 Bit T4
* Negatives: 35 s
* Positives: 38 s

### 8 Bit T4
* Negatives: 23 s
* Positives: 23 s

### Full Res L4
* Negatives: 12.5 s
* Positives: 12.5 s


In [1]:
# https://huggingface.co/docs/transformers/v4.43.4/quantization/bitsandbytes?bnb=4-bit
!pip install accelerate transformers --upgrade -q
!pip install bitsandbytes>=0.39.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 77.5 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from IPython.display import Markdown

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in ca

In [5]:
!nvidia-smi

Tue Aug 20 14:54:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

***note:*** execute in a termial 'watch -n 0.5 nvidia-smi' to see the GPU usage and when the model is loaded onto it

In [24]:
# kind = 'Lllama_3.1_8B_4bit'
kind = 'Lllama_3.1_8B_8bit'
# kind = 'Lllama_3.1_8B_16bit'

In [25]:
%%time

from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

torch_dtype = None
quantization_config = None

if kind == 'Lllama_3.1_8B_4bit':
  quantization_config = BitsAndBytesConfig(load_in_8bit=True)
elif kind == 'Lllama_3.1_8B_8bit':
  quantization_config = BitsAndBytesConfig(load_in_4bit=True)
else:
  torch_dtype = torch.bfloat16

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    torch_dtype=torch_dtype,
    device_map="cuda",
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: user 23.9 s, sys: 10.4 s, total: 34.3 s
Wall time: 12.9 s


In [26]:
!nvidia-smi

Tue Aug 20 14:58:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0              30W /  70W |  14639MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [27]:
positive = [
  "With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.",
  "The socio-medical prerequisites for the prescribed aid supply have been met.",
  "Everyday relevant usage benefits have been determined.",
  "Socio-medical indication for the aid is confirmed.",
  "Contraindications have been excluded; there are no contraindications for the use of the requested aid."
]

In [28]:
negative = [
  "No specific findings can be derived from the diagnosis currently named as the basis for the regulation.",
  "According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.",
  "A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appropriate and therefore necessary has not been transmitted.",
  "From an overall view of the information available here, it cannot be seen how the supply of the insured with the product could be justified, nor can the safety of such a supply be confirmed.",
  "A medical justification for why a product not listed in the directory of aids should be used in the present case has not been transmitted."
]

In [29]:
assessment = negative[0]
# assessment = positive[0]

In [30]:
%%time

messages = [
    {"role": "system", "content": "You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal."},
    {"role": "user", "content": f'''
What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

# Assessment
{assessment}

'''},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=False
)
response = outputs[0][input_ids.shape[-1]:]
Markdown(tokenizer.decode(response, skip_special_tokens=True))

CPU times: user 5.19 s, sys: 227 ms, total: 5.42 s
Wall time: 5.4 s


No

The assessment indicates that no specific findings can be derived from the diagnosis, implying that there is insufficient evidence or information to support a positive or negative recommendation. This suggests that the regulation is not adequately supported by the diagnosis, and further investigation or information is required.

In [31]:
def eval_assessment(assessment):
  messages = [
    {"role": "system", "content": "You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal."},
    {"role": "user", "content": f'''
What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

# Assessment
{assessment}

'''},
]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=512,
      eos_token_id=terminators,
      pad_token_id=tokenizer.eos_token_id,
      do_sample=False
  )
  response = outputs[0][input_ids.shape[-1]:]
  result = tokenizer.decode(response, skip_special_tokens=True)
  if result.startswith("Yes"):
    return "Positive", result
  elif result.startswith("No"):
    return "Negative", result
  else:
    return "Neutral", result

## Negative

In [32]:
%%time

negative_results = []
negative_explanations = []

for assessment in negative:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  negative_results.append(result)
  negative_explanations.append(explanation)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: No specific findings can be derived from the diagnosis currently named as the basis for the regulation.
Negative: No

The assessment indicates that no specific findings can be derived from the diagnosis, implying that there is insufficient evidence or information to support a positive or negative recommendation. This suggests that the regulation is not adequately supported by the diagnosis, and further investigation or information is required.
-----
Assessment: According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.
Negative: No

The assessment indicates that the insured has already been provided with the functional product requested, which suggests that the service is not applicable or is redundant in this case.
-----
Assessment: A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient an

## Positive

In [33]:
%%time

positive_results = []
positive_explanations = []

for assessment in positive:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  positive_results.append(result)
  positive_explanations.append(explanation)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.
Negative: No

The assessment suggests that the need for compensation to ensure the basic need is conceivable, but it does not provide a clear conclusion or recommendation. It implies that further evaluation or investigation is required to determine the actual need for compensation.
-----
Assessment: The socio-medical prerequisites for the prescribed aid supply have been met.
Positive: Yes

The socio-medical prerequisites for the prescribed aid supply have been met, indicating that the individual's medical and social circumstances justify the provision of the aid. This suggests that a positive recommendation is given, as the requirements for the aid supply have been fulfilled.
-----
Assessment: Everyday relevant usage benefits have been determined.
Negative: No

The assessment only determines everyday relevant usage benefits, but does not provide a comprehensive evaluation of th

In [34]:
!nvidia-smi

Tue Aug 20 14:59:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   80C    P0              68W /  70W |  15101MiB / 15360MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [35]:
import pandas as pd

df = pd.DataFrame({
    'assesment': negative + positive,
    'y_true': ['Negative'] * len(negative) + ['Positive'] * len(positive),
    'y_hat': negative_results + positive_results,
    'explanation': negative_explanations + positive_explanations
})
df

,assesment,y_true,y_hat,explanation
0,No specific findings can be derived from the d...,Negative,Negative,No\n\nThe assessment indicates that no specifi...
1,According to the service extracts from the hea...,Negative,Negative,No\n\nThe assessment indicates that the insure...
2,A medically comprehensible explanation as to w...,Negative,Negative,No\n\nJustification: The assessment indicates ...
3,From an overall view of the information availa...,Negative,Negative,No\n\nThe assessment indicates that the supply...
4,A medical justification for why a product not ...,Negative,Negative,No\n\nJustification: The absence of a medical ...
5,"With the diagnosis named here, the need for co...",Positive,Negative,No\n\nThe assessment suggests that the need fo...
6,The socio-medical prerequisites for the prescr...,Positive,Positive,Yes\n\nThe socio-medical prerequisites for the...
7,Everyday relevant usage benefits have been det...,Positive,Negative,No\n\nThe assessment only determines everyday ...
8,Socio-medical indication for the aid is confir...,Positive,Positive,Yes\n\nThe socio-medical indication for the ai...
9,Contraindications have been excluded; there ar...,Positive,Positive,Yes\n\nThe assessment indicates that there are...


In [36]:
df.to_excel(f'results-{kind}.xlsx', index=False)

In [37]:
!ls -l

total 24
-rw-r--r-- 1 root root  161 Aug 20 14:53 '=0.39.0'
-rw-r--r-- 1 root root 6333 Aug 20 14:57  results-Lllama_3.1_8B_4bit.xlsx
-rw-r--r-- 1 root root 6350 Aug 20 14:59  results-Lllama_3.1_8B_8bit.xlsx
drwxr-xr-x 1 root root 4096 Aug 16 13:21  sample_data
